In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

In [44]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
sys.path.append("/Users/Codes/Bitcoin_trader")

In [45]:
from positions import Positions
from order_class import BuyOrder, SellOrder

In [46]:
df = pd.read_parquet('../data/btcusd_preprcd_with_regimes.parquet')
df = df[df['smoothed_regimes'] == 1]
df.dropna(inplace=True)

In [47]:
def preprocess_data(train_df):
    # Log-transform prices
    for col in ['open', 'high', 'low', 'close']:
        train_df[f'{col}_log_return'] = np.log(train_df[col] / train_df[col].shift(1))
    
    # Standard scaling for MACD and CCI
    std_cols = ['MACD_Histogram', 'CCI']
    std_scaler = StandardScaler().fit(train_df[std_cols])
    train_df[std_cols] = std_scaler.transform(train_df[std_cols])
    
    # MinMax for RSI (focus on 30–70 range)
    # train_df['RSI'] = np.clip(train_df['RSI'], 30, 70)
    # rsi_scaler = MinMaxScaler(feature_range=(0, 1)).fit(train_df[['RSI']])
    # train_df['RSI'] = rsi_scaler.transform(train_df[['RSI']])
    
    train_df.drop(columns=['open', 'high', 'low', 'close', 'smoothed_regimes'], inplace=True)
    train_df.dropna(inplace=True)
    return train_df

In [48]:
df = preprocess_data(df)
sc = StandardScaler()
df['predict_trend'] = sc.fit_transform(df[['predict_trend']])

In [49]:
y = df['predict_trend']
x = df.drop(columns=['predict_trend'])

In [50]:
from keras.metrics import MeanSquaredError
import tensorflow as tf

custom_objects = {'mse': MeanSquaredError()}

model_n = tf.keras.models.load_model('../models/sideways_market_model2.h5', custom_objects=custom_objects)

y_pred = model_n.predict(x)

2025-03-18 22:57:00.921967: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Failed to deserialize the `graph_buf`.


8059/8059 ━━━━━━━━━━━━━━━━━━━━ 214s 26ms/step


2025-03-18 23:00:34.308432: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Failed to deserialize the `graph_buf`.


In [51]:
y_pred = sc.inverse_transform(np.array(y_pred).reshape(-1, 1))
y = sc.inverse_transform(np.array(y).reshape(-1, 1))

In [64]:
threshold = 500
signals_pred = np.where(y_pred > threshold, 1, np.where(y_pred < -threshold, -1, 0)).astype(int)
signals_test = np.where(y > threshold, 1, np.where(y < -threshold, -1, 0)).astype(int)

In [71]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, classification_report

# Filter out the 0 class
non_zero_indices = signals_test != 0
filtered_signals_test = signals_test[non_zero_indices]
filtered_signals_pred = signals_pred[non_zero_indices]

# Calculate classification metrics for -1 and 1 classes only
print("Classification Report (Excluding Class 0):")
print(classification_report(filtered_signals_test, filtered_signals_pred, labels=[-1, 1]))

# Calculate accuracy for -1 and 1 classes only
filtered_accuracy = accuracy_score(filtered_signals_test, filtered_signals_pred)
print(f"Accuracy (Excluding Class 0): {filtered_accuracy}")

Classification Report (Excluding Class 0):
              precision    recall  f1-score   support

          -1       0.43      0.10      0.16      1232
           1       0.53      0.25      0.34      1503

   micro avg       0.50      0.18      0.27      2735
   macro avg       0.48      0.17      0.25      2735
weighted avg       0.49      0.18      0.26      2735

Accuracy (Excluding Class 0): 0.17989031078610604


In [65]:
# stop_loss = 0.2%
# take_profit = 0.6%

price = y
signals = signals_pred

In [66]:
def backtest(signals, price, stoploss, takeprofit):
    positions = Positions()
    for i in range(250000, len(signals)):
        positions.check_positions(price[i])
        if signals[i] == 1:
            positions.add_position(BuyOrder(price[i], 1, stoploss=stoploss, takeprofit=takeprofit))
        elif signals[i] == -1:
            positions.add_position(SellOrder(price[i], 1, stoploss=stoploss, takeprofit=takeprofit))
    print(positions.get_profit_count())
    print(positions.get_loss_count())
    print(positions.get_success_rate())
    signal_rate = (positions.get_profit_count()+positions.get_loss_count())/len(signals)
    print(signal_rate)
    


In [67]:
stloss = [0.001, 0.002, 0.003, 0.004, 0.005]
takeprofit = [tp*3 for tp in stloss]

In [68]:
for i in range(len(stloss)):
    st = stloss[i]
    tp = takeprofit[i]
    print(f"Stop loss: {st}")
    print(f"Take profit: {tp}")
    backtest(signals, price, stoploss=st, takeprofit=tp)
    print("\n")

Stop loss: 0.001
Take profit: 0.003
292
310
0.4850498338870432
0.002334355744958063


Stop loss: 0.002
Take profit: 0.006
292
310
0.4850498338870432
0.002334355744958063


Stop loss: 0.003
Take profit: 0.009000000000000001
292
310
0.4850498338870432
0.002334355744958063


Stop loss: 0.004
Take profit: 0.012
292
310
0.4850498338870432
0.002334355744958063


Stop loss: 0.005
Take profit: 0.015
292
310
0.4850498338870432
0.002334355744958063




In [69]:
# print(positions.profit_loss)
